In [1]:
import pandas as pd
import requests
import io
import re
import os
import asyncio
import random
from yahoo_fin.stock_info import get_data
from functools import partial
from bs4 import BeautifulSoup
from IPython.core.debugger import set_trace
from urllib.request import urlretrieve

In [132]:
doc = requests.get('https://www.sslproxies.org', headers={'User-Agent':ua.random})
soup = BeautifulSoup(doc.text, 'html.parser')
proxies = pd.read_html(doc.text, attrs={'id':'proxylisttable'}, converters={'Port': str})[0].dropna()[['IP Address','Port']]
proxies = list((proxies['IP Address'] + ':' + proxies['Port']).values)

In [64]:
def etf_alloc(*etfs):
    
    def _name(df):
        name = df.index.name
        if name.lower() != 'region':
            return name
        elif df.index.str.contains('america|asia|europe|africa|middle', case=False).any():
            return name
        else:
            return 'Market Tier'
    
    def _df(df, etf):
        return pd.DataFrame({etf.upper():df.dropna().Percentage.str.rstrip('%').astype('float')})

    
    async def get_tables(etf):
        url = 'https://etfdb.com/etf/' + etf
        #prx = {'https': proxies.pop(0)}
        #referer = 'https://google.com'        
        #headers = {'User-Agent': ua.random}
        #requests_get = partial(requests.get, headers=headers)
        #response = await loop.run_in_executor(None, requests_get, url)
        
        await loop.run_in_executor(None, urlretrieve, url, 'etfdb/' + etf + '.html')
        
        tables = pd.read_html('etfdb/' + etf + '.html', attrs={'class':'chart base-table'}, index_col=0, flavor=['lxml', 'bs4'])
        return {_name(df):_df(df, etf) for df in tables}


    async def main():
        fts = [asyncio.ensure_future(get_tables(etf)) for etf in etfs]
        return await asyncio.gather(*fts)

    
    asyncio.set_event_loop(asyncio.new_event_loop())
    loop = asyncio.get_event_loop()
    
    try:
        # 다음 코드를 주피터에서 돌리려면, tornado를 downgrade 해야함
        # pip install tornado==4.5.3
        res = loop.run_until_complete(main())       

        etfs = {
            k:pd.concat([dic[k] for dic in res], axis=1, sort='False').fillna(0) for k in res[0]
        }

        out = pd.concat(etfs, axis=0)
    
    except Exception as ex:
        out = ex
    
    finally:
        loop.close()
        
    return out

In [2]:
%%time
etfs = [
    'ACWI', 
    'SPY',
    'QQQ',
    'XLB',
    'XLY',
    'XLP',
    'XLE',
    'XLF',
    'XLV',
    'XLI',
    'IYR',
    'XLK',
    'IYZ', 
    'XLU',
    'EWC',
    'ILF',
    'EWZ',
    'IEV',
    'EWU',
    'EWP',
    'EWG',
    'EWI',
    'RSX',
    'EWQ',
    'EWY', 
    'EWJ',
    'MCHI',
    'EWS',
    'EWA',
    'EWH',
    'EWM',
    'EWT',
    'VNM',
    'EIDO',
    'INDA',
    'EPHE',
    'EFA',
    'EEM',
    'BKF'
]

#etfs = etf_alloc(*etfs)
#etfs = etf_alloc('ACWI','MTUM','VLUE','QUAL','XLB')#,'XLY','XLP','XLE','XLF','XLV','XLI','IYR'); etfs

Wall time: 0 ns


In [155]:
etfs

ACWI    SPY    QQQ     XLB    XLY  \
Asset       Common equity              98.06  99.92  99.97  100.00  99.98   
            ETF Cash Component          1.27   0.08   0.03    0.00   0.02   
            Preferred stock             0.67   0.00   0.00    0.00   0.00   
Country     Argentina                   0.03   0.00   0.21    0.00   0.00   
            Australia                   2.13   0.00   0.00    0.00   0.00   
            Austria                     0.08   0.00   0.00    0.00   0.00   
            Belgium                     0.31   0.00   0.00    0.00   0.00   
            Brazil                      0.81   0.00   0.00    0.00   0.00   
            Canada                      3.08   0.00   0.00    0.00   0.00   
            Chile                       0.11   0.00   0.00    0.00   0.00   
            China                       3.39   0.00   1.37    0.00   0.00   
            Colombia                    0.05   0.00   0.00    0.00   0.00   
            Czech Republic              0.02   0.00   0.00    0.00   0.00   
            Denmark                     0.55   0.00   0.00    0.00   0.00   
            Egypt                       0.01   0.00   0.00    0.00   0.00   
            Finland                     0.43   0.00   0.00    0.00   0.00   
            France                      3.03   0.00   0.00    0.00   0.00   
            Germany                     2.74   0.00   0.78    0.00   0.00   
            Greece                      0.02   0.00   0.00    0.00   0.00   
            Hong Kong                   1.05   0.00   0.00    0.00   0.00   
            Hungary                     0.05   0.00   0.00    0.00   0.00   
            India                       0.92   0.00   0.00    0.00   0.00   
            Indonesia                   0.20   0.00   0.00    0.00   0.00   
            Ireland                     0.71   0.93   0.00    0.00   0.94   
            Israel                      0.16   0.00   0.23    0.00   0.00   
            Italy                       0.67   0.00   0.00    0.00   0.00   
            Japan                       7.12   0.00   0.00    0.00   0.00   
            Liechtenstein               0.01   0.00   0.00    0.00   0.00   
            Luxembourg                  0.08   0.00   0.00    0.00   0.00   
            Malaysia                    0.25   0.00   0.00    0.00   0.00   
...                                      ...    ...    ...     ...    ...   
Market Cap  Large cap                  89.86  91.90  97.88   76.82  85.41   
            Micro cap                   0.01   0.00   0.00    0.00   0.00   
            Mid cap                     8.53   7.92   2.09   23.18  14.18   
            Small cap                   0.34   0.10   0.00    0.00   0.39   
            Unknown                     0.00   0.00   0.00    0.00   0.00   
Market Tier Developed Markets (ex-US)  36.30   1.89   2.25   14.49   1.34   
            Emerging Markets            9.21   0.00   1.58    0.00   0.00   
            Frontier Markets            0.03   0.00   0.00    0.00   0.00   
            Not-designated              0.01   0.00   0.00    0.00   0.00   
            United States              54.44  98.11  96.17   85.51  98.66   
Region      Africa                      0.77   0.00   0.00    0.00   0.00   
            Asia                        7.90   0.00   1.37    0.00   0.00   
            Asia-Pacific               10.88   0.00   0.00    0.00   0.00   
            Europe                     21.26   1.89   2.02   14.49   1.34   
            Latin America               1.31   0.00   0.21    0.00   0.00   
            Middle East                 0.36   0.00   0.23    0.00   0.00   
            North America              57.53  98.11  96.17   85.51  98.66   
Sector      Basic Materials             4.32   2.30   0.00   85.51   0.00   
            Communications              8.89  10.19  22.68    0.00   0.00   
            Consumer, Cyclical         10.02   9.72  15.53    0.00  95.60   
            Consumer, Non-Cyclical  

In [26]:
%%time
import fix_yahoo_finance as yf
data = yf.download(etfs, start="2010-01-01", end="2017-04-30")

[*********************100%***********************]  39 of 39 downloaded
Wall time: 22.6 s


In [27]:
data

Open                                                         \
                 ACWI        BKF        EEM        EFA       EIDO       EPHE   
Date                                                                           
2009-12-31  42.490002  45.980000  41.630001  55.919998        NaN        NaN   
2010-01-04  43.110001  47.160000  42.180000  56.419998        NaN        NaN   
2010-01-05  43.340000  47.770000  42.910000  56.869999        NaN        NaN   
2010-01-06  43.320000  47.639999  43.080002  56.680000        NaN        NaN   
2010-01-07  43.419998  47.610001  42.849998  56.639999        NaN        NaN   
2010-01-08  43.540001  47.830002  42.880001  56.900002        NaN        NaN   
2010-01-11  43.779999  48.139999  43.450001  57.830002        NaN        NaN   
2010-01-12  43.549999  47.290001  42.610001  57.110001        NaN        NaN   
2010-01-13  43.360001  47.279999  42.590000  57.279999        NaN        NaN   
2010-01-14  43.689999  47.000000  42.520000  57.500000        NaN        NaN   
2010-01-15  43.700001  46.799999  42.320000  57.509998        NaN        NaN   
2010-01-19  43.200001  46.660000  42.080002  56.880001        NaN        NaN   
2010-01-20  43.270000  46.340000  42.110001  56.369999        NaN        NaN   
2010-01-21  42.730000  45.459999  41.549999  56.040001        NaN        NaN   
2010-01-22  41.980000  43.619999  40.220001  54.700001        NaN        NaN   
2010-01-25  41.570000  43.759998  40.130001  54.860001        NaN        NaN   
2010-01-26  41.310001  42.799999  39.130001  54.049999        NaN        NaN   
2010-01-27  40.980000  42.099998  38.790001  53.880001        NaN        NaN   
2010-01-28  41.549999  42.209999  39.320000  54.189999        NaN        NaN   
2010-01-29  40.689999  42.270000  39.150002  53.320000        NaN        NaN   
2010-02-01  40.410000  41.669998  38.639999  53.200001        NaN        NaN   
2010-02-02  41.200001  42.540001  39.389999  53.820000        NaN        NaN   
2010-02-03  41.279999  43.470001  39.639999  53.980000        NaN        NaN   
2010-02-04  40.439999  42.450001  38.750000  52.740002        NaN        NaN   
2010-02-05  39.669998  40.330002  37.290001  51.290001        NaN        NaN   
2010-02-08  39.480000  40.369999  37.180000  50.970001        NaN        NaN   
2010-02-09  39.919998  41.029999  37.730000  51.590000        NaN        NaN   
2010-02-10  40.070000  41.470001  38.040001  51.770000        NaN        NaN   
2010-02-11  39.779999  41.570000  38.189999  51.540001        NaN        NaN   
2010-02-12  39.950001  41.349998  38.139999  51.299999        NaN        NaN   
...               ...        ...        ...        ...        ...        ...   
2017-03-17  63.619999  36.619999  39.599998  62.180000  25.570000  34.650002   
2017-03-20  63.580002  36.580002  39.740002  62.250000  25.620001  34.410000   
2017-03-21  63.790001  36.959999  40.150002  62.580002  25.680000  34.650002   
2017-03-22  62.830002  36.060001  39.480000  61.619999  25.620001  33.980000   
2017-03-23  62.939999  36.180000  39.560001  61.730000  25.850000  34.169998   
2017-03-24  63.090000  36.380001  39.680000  62.110001  25.889999  34.180000   
2017-03-27  62.689999  36.060001  39.549999  62.090000  25.809999  34.009998   
2017-03-28  63.070000  36.360001  39.669998  62.389999  25.660000  34.459999   
2017-03-29  63.150002  36.380001  39.720001  62.290001  25.900000  34.400002   
2017-03-30  63.419998  36.490002  39.720001  62.430000  25.959999  34.430000   
2017-03-31  63.240002  36.240002  39.419998  62.110001  25.730000  34.180000   
2017-04-03  63.310001  36.340000  39.509998  62.189999  26.040001  34.310001   
2017-04-04  63.090000  36.580002  39.529999  61.930000  26.250000  34.889999   
2017-04-05  63.430000  36.840000  39.799999  62.130001  26.420000  35.380001   
2017-04-06  63.139999  36.540001  39.470001  61.880001  26.299999  35.389999   
2017-04-07  63.070000  36.430000  39.360001  61.750000  26.219999  35.680000   
2017-04-10  63.09

In [95]:
from yahoo_fin.stock_info import get_data

In [29]:
%%time
etfs = ['spy','acwi','mtum','qual']
start = '2017-12-31'
end = '2018-12-31'

pr = {}
for etf in etfs:
    pr[etf.upper()] = get_data(etf, start_date=start, end_date=end).adjclose
    #pr.append(get_data(etf, start_date=start, end_date=end).adjclose)

Wall time: 9.44 s


In [22]:
df = pd.concat(pr, axis=1).fillna(method='ffill');df

,ACWI,MTUM,SPY
date,,,
2018-01-02,71.268372,102.736465,263.759949
2018-01-03,71.630737,103.764420,265.428253
2018-01-04,72.130219,104.634232,266.546997
2018-01-05,72.600311,105.474380,268.323273
2018-01-08,72.668869,105.770912,268.813934
2018-01-09,72.757011,106.166283,269.422363
2018-01-10,72.590523,106.057556,269.010223
2018-01-11,73.089996,106.779106,270.972900
2018-01-12,73.648224,107.797173,272.739380


In [110]:
def etf_price(*etfs, start=None, end=None):
    
    async def get_price(etf):
        get_data_partial = partial(get_data, start_date=start, end_date=end)
        pr = await loop.run_in_executor(None, get_data_partial, etf)
        return etf, pr.adjclose
    
    async def main():
        fts = [asyncio.ensure_future(get_price(etf)) for etf in etfs]
        return await asyncio.gather(*fts)
    
    asyncio.set_event_loop(asyncio.new_event_loop())
    loop = asyncio.get_event_loop()
    
    try:
        # 다음 코드를 주피터에서 돌리려면, tornado를 downgrade 해야함
        # pip install tornado==4.5.3
        res = loop.run_until_complete(main())        
        out = pd.concat(dict(res), axis=1, sort=True).fillna(method='ffill')
    
    except Exception as ex:
        out = ex
    
    finally:
        loop.close()
        
    out = out[list(etfs)]
    out.columns = out.columns.str.upper()
    return out

In [111]:
%%time
etfs = ['spy','acwi','mtum','qual']
y_period = 3
end = pd.Timestamp.today()
start = end - pd.DateOffset(years=y_period)
prices = etf_price(*etfs, start=start, end=end)

Wall time: 2.3 s


In [112]:
prices

,SPY,ACWI,MTUM,QUAL
date,,,,
2016-01-28,178.218582,48.566456,66.648605,57.323166
2016-01-29,182.563065,49.608170,67.920296,58.705257
2016-02-01,182.497055,49.514324,68.286415,58.752911
2016-02-02,179.208099,48.463223,67.313362,57.771152
2016-02-03,180.282440,48.838619,66.610069,58.190552
2016-02-04,180.565170,48.979385,65.935699,58.304924
2016-02-05,177.125351,48.162910,63.960701,57.246925
2016-02-08,174.741058,47.374584,62.505951,56.655952
2016-02-09,174.750473,47.111813,62.843151,56.798923


In [67]:
cov = prices.pct_change().cov(min_periods=250)*250; cov

,ACWI,MTUM,QUAL,SPY
ACWI,0.016123,0.016552,0.014718,0.015502
MTUM,0.016552,0.023064,0.017212,0.017862
QUAL,0.014718,0.017212,0.015508,0.015616
SPY,0.015502,0.017862,0.015616,0.016355


In [70]:
cov.iloc[1,1]**0.5

0.1518675643592027

In [92]:
ETFDB_BASE_URL = 'https://etfdb.com/etf/'
DIR_ETFDB = 'etfdb'

def etf_alloc(*etfs, at=None):
    cwd = os.getcwd()
    dir_target = os.path.join(cwd, DIR_ETFDB, at)

    def _name(df):
        name = df.index.name
        if name.lower() != 'region':
            return name
        elif df.index.str.contains('america|asia|europe|africa|middle', case=False).any():
            return name
        else:
            return 'Market Tier'

    def _df(df, etf):
        return pd.DataFrame({etf.upper():df.dropna().Percentage.str.rstrip('%').astype('float')})


    def _tables(etf):
        file = os.path.join(dir_target, etf + '.html')
        tables = pd.read_html(file, attrs={'class':'chart base-table'}, index_col=0)
        return {_name(df):_df(df, etf) for df in tables}


    if not os.path.exists(dir_target):
        os.makedirs(dir_target)
    
    for etf in etfs:
        file = os.path.join(dir_target, etf.upper() + '.html')
        if not os.path.exists(file):
            urlretrieve(ETFDB_BASE_URL + etf, file)

    tables = []
    for filename in os.listdir(dir_target):
        if filename.endswith('.html'):
            tables.append(_tables(filename.split('.')[-2]))

    tables_dict = {
        k:pd.concat([dic[k] for dic in tables], axis=1, sort='False').fillna(0) for k in tables[0]
    }

    return pd.concat(tables_dict, axis=0)  

In [94]:
%%time
etf_alloc(*etfs, at='2019-01-30')

Wall time: 948 ms


ACWI    EWC    EWG    EWI    EWP  \
Asset       Common equity              98.06  99.49  93.22  98.66  99.52   
            ETF Cash Component          1.27   0.51   0.78   0.22   0.48   
            Preferred stock             0.67   0.00   6.00   1.13   0.00   
Country     Argentina                   0.03   0.00   0.00   0.00   0.00   
            Australia                   2.13   0.00   0.00   0.00   0.00   
            Austria                     0.08   0.00   0.00   0.00   0.00   
            Belgium                     0.31   0.00   0.00   0.00   0.00   
            Brazil                      0.81   0.00   0.00   0.00   0.00   
            Canada                      3.08  96.67   0.00   0.00   0.00   
            Chile                       0.11   0.00   0.00   0.00   0.00   
            China                       3.39   0.00   0.00   1.16   0.00   
            Colombia                    0.05   0.00   0.00   0.00   0.00   
            Cyprus                      0.00   0.00   0.00   0.00   0.00   
            Czech Republic              0.02   0.00   0.00   0.00   0.00   
            Denmark                     0.55   0.00   0.00   0.00   0.00   
            Egypt                       0.01   0.00   0.00   0.00   0.00   
            Finland                     0.43   0.00   0.00   0.00   0.00   
            France                      3.03   0.00   0.00   0.00   0.00   
            Germany                     2.74   0.00  97.47   0.00   1.56   
            Greece                      0.02   0.00   0.00   0.00   0.00   
            Hong Kong                   1.05   0.00   0.00   0.00   0.00   
            Hungary                     0.05   0.00   0.00   0.00   0.00   
            India                       0.92   0.00   0.00   0.00   0.00   
            Indonesia                   0.20   0.00   0.00   0.00   0.00   
            Ireland                     0.71   0.00   0.00   0.00   0.00   
            Israel                      0.16   0.00   0.00   0.00   0.00   
            Italy                       0.67   0.00   0.00  85.14   2.92   
            Japan                       7.12   0.00   0.00   0.00   0.00   
            Liechtenstein               0.01   0.00   0.00   0.00   0.00   
            Luxembourg                  0.08   0.00   0.56   0.00   0.00   
...                                      ...    ...    ...    ...    ...   
Market Cap  Large cap                  89.86  92.85  79.84  95.32  74.87   
            Micro cap                   0.01   0.00   0.00   0.00   0.00   
            Mid cap                     8.53   6.64  15.99   4.46  24.60   
            Small cap                   0.34   0.00   2.93   0.00   0.00   
            Unknown                     0.00   0.00   0.45   0.00   0.05   
Market Tier Developed Markets (ex-US)  36.30  97.22  99.22  98.62  99.52   
            Emerging Markets            9.21   0.00   0.00   1.16   0.00   
            Frontier Markets            0.03   0.00   0.00   0.00   0.00   
            Not-designated              0.01   0.00   0.00   0.00   0.00   
            United States              54.44   2.78   0.78   0.22   0.48   
Region      Africa                      0.77   0.00   0.00   0.00   0.00   
            Asia                        7.90   0.00   0.00   1.16   0.00   
            Asia-Pacific               10.88   0.00   0.00   0.00   0.00   
            Europe                     21.26   0.55  99.22  98.62  99.52   
            Latin America               1.31   0.00   0.00   0.00   0.00   
            Middle East                 0.36   0.00   0.00   0.00   0.00   
            North America              57.53  99.45   0.78   0.22   0.48   
Sector      Basic Materials             4.32   7.89  10.04   0.00   0.00   
            Communications              8.89   3.18   6.15   3.32   8.63   
            Consumer, Cyclical         10.02   4.31  17.84  12.66   4.01   
            Consumer, Non-Cyclical      7.98   4.37   3.32   2.20   0.00   
            

In [73]:
etfs = [
    'ACWI', 
    'SPY',
    'QQQ',
    'XLB',
    'XLY',
    'XLP',
    'XLE',
    'XLF',
    'XLV',
#     'XLI',
#     'IYR',
#     'XLK',
#     'IYZ', 
#     'XLU',
#     'EWC',
#     'ILF',
#     'EWZ',
#     'IEV',
#     'EWU',
#     'EWP',
#     'EWG',
#     'EWI',
#     'RSX',
#     'EWQ',
#     'EWY', 
#     'EWJ',
#     'MCHI',
#     'EWS',
#     'EWA',
#     'EWH',
#     'EWM',
#     'EWT',
#     'VNM',
#     'EIDO',
#     'INDA',
#     'EPHE',
#     'EFA',
#     'EEM',
#     'BKF'
]